In [2]:
import os
from dotenv import load_dotenv
from pprint import pprint

if load_dotenv(".env"):
    # for local environment
    OPENAI_KEY = os.getenv("OPENAI_API_KEY")
    GEMINI_KEY = os.getenv("GEMINI_API_KEY")
    GROQ_KEY = os.getenv("GROQ_API_KEY") 
    TAVILY_KEY = os.getenv("TAVILY_API_KEY")
    GEMINI_MODEL = os.getenv('GEMINI_MODEL_NAME')
    GROQ_MODEL = os.getenv("GROQ_MODEL_NAME")
    
#else:
    # for streamlit community cloud environment
#    OPENAI_KEY = st.secrets["OPENAI_API_KEY"]

In [3]:
# Set up Tavily Search Agent using Tavily API
from langchain_tavily import TavilySearch
from typing import List, Literal,Dict

tavily_search_tool = TavilySearch(max_results=5, topic='general', include_answer=False,
                               include_raw_content=True, search_depth='basic') #Note that 'search_depth' cannot be called during invoke.

In [4]:
from langchain.chat_models import init_chat_model

llama_llm = init_chat_model(model=GROQ_MODEL, model_provider="groq", temperature=0, top_p = 1, max_tokens = 1024)
llama_llm

d:\Data Science\DSProjects\MergerScan\venv\Lib\site-packages\pydantic\main.py:253: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001773055DA90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001773055E660>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={'top_p': 1}, groq_api_key=SecretStr('**********'), max_tokens=1024)

In [6]:
# Creating agent with web search
from langgraph.prebuilt import create_react_agent

sys_prompt = """You are a helpful reaserch assistant. You will be given a query and you can use the Tavily search tool to search the web for information to supplement your internal
knowledge base. If you use the Tavily search tool, analyse and use the most relevant content from the returned results to answer the query. If you think that there is no relevant 
content in the returned results and you do not have the answer in your knowledge base, respond "Sorry, I do not have the answer. Try rephrasing your query".
If you utilise content from the returned results, provide the corresponding URLs in your citation at the end of the response."""


agent = create_react_agent(model=llama_llm, tools=[tavily_search_tool], prompt=sys_prompt)

user_input = "Does Typerwriter Ascend Ltd have any operation in Singapore? Include only sources with domain gov.sg. Exclude sources from facebook and instagram"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Does Typerwriter Ascend Ltd have any operation in Singapore? Include only sources with domain gov.sg. Exclude sources from facebook and instagram
================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc985vv5m)
 Call ID: fc985vv5m
  Args:
    exclude_domains: ['facebook.com', 'instagram.com']
    include_domains: ['gov.sg']
    query: Typerwriter Ascend Ltd Singapore
================================= Tool Message =================================
Name: tavily_search

{"query": "Typerwriter Ascend Ltd Singapore", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.cccs.gov.sg/media-and-events/newsroom/announcements-and-media-releases/cccs-clears-proposed-acquisition-of-global-sea-containers-limited-by-typewriter-ascend-ltd/", "title": "CCCS Clears Proposed Acquisition of Global Sea Containers ...", "content": "C

In [ ]:
result = tavily_search_tool.invoke({"query": "Does Typerwriter Ascend Ltd have any operation in Singapore?","exclude_domains":["facebook.com","instagram.com"],
                             "include_domains":["*.com.sg", "*.gov.sg"],"time_range": "day","day":14})
result

{'query': 'Does Typerwriter Ascend Ltd have any operation in Singapore?',
 'follow_up_questions': None,
 'answer': 'Yes, Typewriter Ascend Ltd has operations in Singapore. The company is involved in a proposed acquisition of Global Sea Containers Limited (Seaco), which was reviewed by the Competition and Consumer Commission of Singapore (CCCS). On July 15, 2025, CCCS accepted a joint application from both companies for a decision under Section 57 of the Competition Act 2004 regarding whether the proposed transaction would infringe competition laws. The CCCS issued its grounds of decision on August 19, 2025, indicating that Typewriter Ascend Ltd operates in the intermodal containers supply market in Singapore and globally. The regulatory review process demonstrates that the company has sufficient business presence in Singapore to warrant competition law scrutiny for its proposed acquisition activities.',
 'images': [],
 'results': [{'url': 'https://isomer-user-content.by.gov.sg/45/f1318

In [11]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_KEY)
response = tavily_client.search(query="Does Typerwriter Ascend Ltd have any operation in Singapore?", max_results=5, topic='general', include_answer='advanced',
                               include_raw_content=True, search_depth='advanced')
print(response)

{'query': 'Does Typerwriter Ascend Ltd have any operation in Singapore?', 'follow_up_questions': None, 'answer': "Yes, Typewriter Ascend Ltd has operations in Singapore. The company was involved in a significant acquisition transaction that required regulatory approval from Singapore's Competition and Consumer Commission of Singapore (CCCS). In July 2025, Typewriter Ascend Ltd, along with Global Sea Containers Limited, submitted a joint application to CCCS for approval of their proposed merger under Singapore's Competition Act 2004. The CCCS cleared this proposed acquisition in August 2025, determining that the transaction would not substantially lessen competition within Singapore's market. The fact that Typewriter Ascend Ltd needed to seek regulatory approval from Singapore authorities and comply with Singapore's competition laws indicates that the company has business operations or interests that fall under Singapore's regulatory jurisdiction, particularly in the intermodal containe